### Attractor Simulation

In [1]:
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(BoolNet))
suppressPackageStartupMessages(require(lhs))

In [2]:
# getwd()
library(BoolNet)
#############################
# Node score : weighted sum #
#############################
getAttrScore <- function(attrs, nodes=NULL) {
  basinSizes <- sapply(attrs$attractors, function(x) x$basinSize)
  node.scores.for.attractor <- sapply(seq_along(attrs$attractors),
                                      function(x) {
                                        attr.seq <- getAttractorSequence(attrs, x)
                                        apply(attr.seq, 2, mean) * basinSizes[x]
                                      })                       
  node.score <- apply(node.scores.for.attractor, 1, sum) / sum(basinSizes)  
  if(!is.null(nodes)) {as.matrix(node.score[nodes])} else {as.matrix(node.score)}
}


############################
# Definition of Phenotypes #
############################
is.no_prolif <- function(x) all(colMeans(x[,'Proliferation',drop=F])<0.5)
is.prolif <- function(x) all(colMeans(x[,'Proliferation',drop=F])>=0.5 & colMeans(x[,'Apoptosis',drop=F])<0.5)
is.apop <- function(x) all(colMeans(x[,'Proliferation',drop=F])<0.5 & colMeans(x[,'Apoptosis',drop=F])>=0.5)
is.arrest <- function(x) all(colMeans(x[,'Proliferation',drop=F])>=0.5 & colMeans(x[,'Apoptosis',drop=F])>=0.5 & colMeans(x[,'CDKN1A',drop=F])>=0.5)

                       
phenotype.function.list <- list(
  Prolif = is.prolif,
  Apop = is.apop,
  NoProlif = is.no_prolif,
  Arrest = is.arrest 
)

getPhenoScore <- function(attrs) {
  basinSizes <- sapply(attrs$attractors, function(x) x$basinSize)
  score <- apply(
    sapply(seq_along(attrs$attractors),
           function(x) {
             attr.seq <- getAttractorSequence(attrs, x)
             #print(attr.seq)
             sapply(phenotype.function.list, function(pf) pf(attr.seq)) * basinSizes[x]
           }), 1, sum
  ) / sum(basinSizes)
  score
}

In [3]:
###################
# Simulation part #
###################
# Load model
boolnetwork <- loadNetwork('./io_relationship/LGC_Network.txt')


# Number of initial conditions
num_init <- 100000; 

## Node perturbations
perturb.list<-list()
perturb.list[['Resistant']] = list(ON=c('KRAS'), OFF=c())

perturb.list[['Res_MEK_OFF']] = list(ON=c('KRAS'), OFF=c('MAP2K1','MAP2K2'))

perturb.list[['Res_PIN1_OFF']] = list(ON=c('KRAS'), OFF=c('PIN1'))
perturb.list[['Res_PIN1_OFF_MEK']] = list(ON=c('KRAS'), OFF=c('MAP2K1','MAP2K2','PIN1'))

perturb.list[['Res_AKT1_OFF']] = list(ON=c('KRAS'), OFF=c('AKT1'))
perturb.list[['Res_AKT1_OFF_MEK']] = list(ON=c('KRAS'), OFF=c('MAP2K1','MAP2K2','AKT1'))

perturb.list[['Res_IRS1_OFF']] = list(ON=c('KRAS'), OFF=c('IRS1'))
perturb.list[['Res_IRS1_OFF_MEK']] = list(ON=c('KRAS'), OFF=c('MAP2K1','MAP2K2','IRS1'))



make_unique <- function(x) {
  x['ON'] <- unique(x['ON'])
  x['OFF'] <- unique(x['OFF'])
  return(x)
}

pert.list <- lapply(perturb.list,make_unique)
length(pert.list)

[1] 8

In [4]:
set.seed(12345)
attr.list <- list()
att.all <- c()
system.time(
  for(i in seq_along(pert.list))  {
    #att.all <- c()
    attrs <- getAttractors(boolnetwork, type="synchronous", method="random", canonical=F, avoidSelfLoops=FALSE,
                           startStates=num_init,
                           genesON=pert.list[[i]]$ON,
                           genesOFF=pert.list[[i]]$OFF)
    attr.list[[names(pert.list)[i]]] <- attrs
    for (p in 1:length(attrs$attractors)){
      b <- attrs$attractors[[p]]$basinSize
      att <- getAttractorSequence(attrs, p)
      basinsize <- c()
      for (j in 1:length(att[,1])){basinsize <- c(basinsize,b)}
      att <- cbind(basinsize,att)
      att.all <- rbind(att.all,att)
    }
    
    #write.table(att.all, file = paste(as.character(i),"_attractor_all.csv"), sep = ",", quote = TRUE, row.names = TRUE)
  }
)

attr.scores <- sapply(attr.list, function(x) getAttrScore(x))
pheno.scores <- sapply(attr.list, function(x) getPhenoScore(x))
rownames(attr.scores) <- boolnetwork$genes
pheno.result <- data.frame(t(data.frame(rbind(pheno.scores,attr.scores))))

   user  system elapsed 
124.631   3.827 128.459 

In [ ]:
# d <- pheno.result[,colnames(pheno.result) %in% c('Apop','Prolif','NoProlif','Arrest','Proliferation','Apoptosis')]
# d <- d[order((d$Apoptosis-d$Proliferation), decreasing = T),]
# d

In [6]:
# # write.csv(pheno.result, 'pheno_result_single_double_100000.csv')
# write.csv(attr.scores, '/data/Gitcode/attr_scores_100000.csv')